# MVP Engenharia de Dados

**Autor:** Roger Luis Giroldo Assunção

**Data:** 21/12/2025

**Matrícula:** 4052025000584

**Dataset:** AI Incident Database (AIID)

# Objetivo



Nos últimos anos, o avanço acelerado das tecnologias digitais — especialmente algoritmos e sistemas de inteligência artificial (IA) — transformou profundamente a forma como vivemos, trabalhamos e interagimos. Esses sistemas se tornaram onipresentes, impulsionando decisões em áreas como saúde, finanças, mobilidade, comunicação e segurança.
Com o crescimento do uso e da complexidade das soluções baseadas em IA, também aumentou nossa dependência delas, ao mesmo tempo em que se ampliou a exposição a riscos associados ao seu funcionamento, às suas limitações e às consequências de eventuais falhas.
Entre esses riscos, destacam‑se impactos diretos sobre a vida humana, possíveis discriminações resultantes de vieses presentes nos dados usados para treinar modelos e a expansão do alcance de erros algorítmicos em escala social.

Diante desse cenário, este trabalho tem como objetivo responder às seguintes questões centrais:

1. Existem incidentes de IA registrados que envolvem morte de pessoas?  
2. Existem incidentes registrados que evidenciem que modelos de IA absorvem problemas de viés social, como racismo, homofobia, intolerância religiosa e outros preconceitos?  
3. Qual é a representatividade desses tipos de incidente dentre os registros existentes?  
4. A taxa de ocorrência de incidentes de IA vem aumentando nos últimos anos?

## Busca pelos dados

Foi realizada uma busca exaustiva por datasets que contivessem potencialmente os dados para responder às perguntas acima e que fossem dados revisados e com metodologia bem definida de coleta e classificação.
Foi encontrado o *AI Incident Database* (AIID) - https://incidentdatabase.ai - com potencial de nos embasar nessa empreitada.

- Licença: CC BY-SA 4.0

- Citação:
> McGregor, S. (2021) Preventing Repeated Real World AI Failures by Cataloging Incidents: The AI Incident Database. In Proceedings of the Thirty-Third Annual Conference on Innovative Applications of Artificial Intelligence (IAAI-21). Virtual Conference.

- Paper: https://arxiv.org/abs/2011.08512

# Preparação

Criação do catálogo e dos esquemas a serem utilizados:

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS mvp
COMMENT 'MVP de Engenharia de Dados';

CREATE SCHEMA IF NOT EXISTS mvp.staging
COMMENT 'Local para downloads e manipulação/exploração inicial';

CREATE SCHEMA IF NOT EXISTS mvp.bronze
COMMENT 'Camada bronze';

CREATE SCHEMA IF NOT EXISTS mvp.silver
COMMENT 'Camada prata';

CREATE SCHEMA IF NOT EXISTS mvp.gold
COMMENT 'Camada ouro';

Todos os imports (python) para o notebook estão consolidados abaixo:

In [0]:
import tarfile
import os
import bson
import json
import glob
from pyspark.sql import functions as F

# Staging

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS mvp.staging.aiid
COMMENT 'Dados do AI Incident Database (AIID)';

## Ingestão

### Download dos dados

Download e extração do archive contendo os dados brutos:

In [0]:
# Download dos dados brutos
dbutils.fs.cp(
    "https://pub-72b2b2fc36ec423189843747af98f80e.r2.dev/backup-20251215100704.tar.bz2",
    "/Volumes/mvp/staging/aiid",
)

# Extrai tudo do archive recém baixado
with tarfile.open(
    "/Volumes/mvp/staging/aiid/backup-20251215100704.tar.bz2", "r:bz2"
) as tar:
    tar.extractall("/Volumes/mvp/staging/aiid/data")

True

In [0]:
# Lista os arquivos extraídos
startpath = "/Volumes/mvp/staging/aiid/data/mongodump_full_snapshot"
print("Arquivos extraídos:\n")
for root, dirs, files in os.walk(startpath):
    level = root.replace(startpath, "", 1).count(os.sep)
    indent = " " * 4 * (level)
    print("{}- {}/".format(indent, os.path.basename(root)))
    subindent = " " * 4 * (level + 1)
    for f in files:
        print("{}- {}".format(subindent, f))

Arquivos extraídos:

- mongodump_full_snapshot/
    - classifications_CSETv0.csv
    - classifications_CSETv1.csv
    - classifications_CSETv1_Annotator-1.csv
    - classifications_CSETv1_Annotator-2.csv
    - classifications_CSETv1_Annotator-3.csv
    - classifications_GMF.csv
    - classifications_MIT.csv
    - duplicates.csv
    - incidents.csv
    - license.txt
    - quickadd.csv
    - reports.csv
    - submissions.csv
    - aiidprod/
        - candidates.bson
        - candidates.metadata.json
        - checklists.bson
        - checklists.metadata.json
        - classifications.bson
        - classifications.metadata.json
        - duplicates.bson
        - duplicates.metadata.json
        - entities.bson
        - entities.metadata.json
        - entity_duplicates.bson
        - entity_duplicates.metadata.json
        - entity_relationships.bson
        - entity_relationships.metadata.json
        - incident_links.bson
        - incident_links.metadata.json
        - incidents.b

### Ingestão e persistência em tabelas

In [0]:
# Função recursiva de conversão explícita bson -> python para tipos que não são compatíveis com spark dataframe
def convert_bson_types(obj):
    if isinstance(obj, dict):
        return {k: convert_bson_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_bson_types(v) for v in obj]
    elif isinstance(obj, bson.ObjectId):
        return str(obj)
    elif isinstance(obj, bson.Int64):
        return int(obj)
    else:
        return obj

Ingestão com Spark e persistência em tabelas Delta:

In [0]:
# Lê os arquivos .bson provenientes do mongodump disponibilizado pelo AIID e salva em Delta Tables
for bson_file in glob.glob(
    "/Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/*.bson"
):
    docs = []
    name = os.path.splitext(os.path.basename(bson_file))[0]

    print("Processando:", bson_file)
    with open(bson_file, "rb") as f:
        for doc in bson.decode_file_iter(f):
            # Regulariza tipos
            doc = convert_bson_types(doc)

            # Elimina espaços dos nomes de atributos
            key_list = list(doc.keys())
            for k in key_list:
                if k.count(" ") > 0:
                    doc[k.replace(" ", "_")] = doc.pop(k)

            # Trata casos especiais em que a inferência de tipos não funciona
            if name == "candidates":
                if "embedding" in doc:
                    doc["embedding"] = json.dumps(doc["embedding"])
            elif name == "checklists":
                if "risks" in doc:
                    doc["risks"] = json.dumps(doc["risks"])
            elif name == "classifications":
                del doc["reports"]
            elif name == "incidents":
                if "embedding" in doc:
                    doc["embedding"] = json.dumps(doc["embedding"])
                if "nlp_similar_incidents" in doc:
                    doc["nlp_similar_incidents"] = json.dumps(
                        doc["nlp_similar_incidents"]
                    )
            elif name == "reports":
                if "embedding" in doc:
                    doc["embedding"] = json.dumps(doc["embedding"])
                if "epoch_date_published" in doc and isinstance(
                    doc["epoch_date_published"], float
                ):
                    try:
                        doc["epoch_date_published"] = int(doc["epoch_date_published"])
                    except:
                        doc["epoch_date_published"] = None
                if "epoch_date_submitted" in doc and isinstance(
                    doc["epoch_date_submitted"], float
                ):
                    try:
                        doc["epoch_date_submitted"] = int(doc["epoch_date_submitted"])
                    except:
                        doc["epoch_date_submitted"] = None
            elif name == "taxa":
                if "field_list" in doc:
                    doc["field_list"] = json.dumps(doc["field_list"])

            docs.append(doc)

    if docs == []:
        continue

    # Converte para dataframe e salva em delta table
    df = spark.createDataFrame(docs)
    df.write.format("delta").mode("overwrite").saveAsTable(f"mvp.staging.{name}")

print("\n✅ Tabelas criadas com sucesso!")

Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/candidates.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/checklists.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/classifications.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/duplicates.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/entities.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/entity_duplicates.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/entity_relationships.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/incident_links.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/incidents.bson
Processando: /Volumes/mvp/staging/aiid/data/mongodump_full_snapshot/aiidprod/migrations.bson
Processando: /Volumes/mvp/staging/aiid/data/mon

A ingestão foi realizada utilizando o backup do database original dispobilizado na fonte, que consiste em um mongodump de um database MongoDB.

A forma mais direta de ingestão foi fazer a leitura dos arquivos .bson gerados pelo dump, deixando que o Spark fizesse as inferências necessárias para a conversão para dataframe.
Essa abordagem adotada é superior a ingerir os dados dos arquivos .csv, visto que os dados no formato bson contêm informação dos tipos e, dessa forma, a migração mantém os tipos da forma mais fiel possível.

Alguns atributos causaram erro de inferência inicialmente. Isso é muito comum ao tentar migrar de um database NoSQL para um com estrutura tabular, como o Delta Lake.

Para contornar alguns erros de inferência, alguns tratamentos prévios foram realizados:
- Objetos originais que continham dados heterogêneos foram convertidos para json strings utilizando json.dumps();
- Atributos contendo apenas objetos vazios, listas vazias ou nulos foram eliminados;
- Dentre outros ajustes de tipos.

# Camada Bronze

Após exploração visual das tabelas através da navegação pela interface gráfica do Databricks, foi possível definir quais tabelas contém dados potencialmente úteis para nossa modelagem e análise, de forma que consigamos obter as respostas às nossas perguntas do objetivo.

Desse modo, a camada bronze consistirá na cópia dessas tabelas identificadas como úteis dentre as tabelas presentes no esquema *staging*.

In [0]:
%sql
CREATE OR REPLACE TABLE mvp.bronze.incidents CLONE mvp.staging.incidents;

CREATE OR REPLACE TABLE mvp.bronze.classifications CLONE mvp.staging.classifications;

CREATE OR REPLACE TABLE mvp.bronze.reports CLONE mvp.staging.reports;

CREATE OR REPLACE TABLE mvp.bronze.entities CLONE mvp.staging.entities;

As tabelas *classifications*, *incidents*, *entities* e *reports* possuem os principais dados de todo o database inicial e foram clonadas para a camada bronze.

Dentre essas tabelas, segue um extrato das duas principais, por ordem de importância, **incidents** e **classifications**.

In [0]:
%sql
SELECT * FROM mvp.bronze.incidents
LIMIT 10

Alleged_deployer_of_AI_system Alleged_developer_of_AI_system Alleged_harmed_or_nearly_harmed_parties _id created_at date description editor_dissimilar_incidents editor_notes editor_similar_incidents editors embedding flagged_dissimilar_incidents implicated_systems incident_id nlp_similar_incidents reports title tsne date_modified epoch_date_modified _id__baas_transaction AllegedDeployerOfAISystem AllegedDeveloperOfAISystem AllegedHarmedOrNearlyHarmedParties List(navya, keolis-north-america) List(navya, keolis-north-america) List(navya, keolis-north-america, bus-passengers) 625763de343edc875fe63a15 2017-11-08T00:00:00.000Z 2017-11-08 A self-driving public shuttle by Keolis North America and Navya was involved in a collision with a human-driven delivery truck in Las Vegas, Nevada on its first day of service. List() List() List(619b47ea5eed5334edfa3bbc, 62970eca16e5e43939c226f7) {"vector": [-0.08961334824562073, 0.07114614546298981, 0.016013426706194878, -0.11896050721406937, 0.06678394973278046, -0.016360925510525703, -0.016785796731710434, 0.023858457803726196, 0.060031261295080185, -0.1496659219264984, 0.001895593828521669, 0.05982673168182373, 0.012797102332115173, -0.09214386343955994, 0.05113914608955383, -0.11387323588132858, -0.07738547772169113, -0.028795281425118446, -0.011059424839913845, -0.10338249802589417, -0.08604574203491211, -0.014408774673938751, 0.04922022297978401, 0.10739009827375412, -0.05844538286328316, 0.018406828865408897, 0.04563510790467262, 0.11527135223150253, -0.018671289086341858, 0.037182509899139404, -0.004360778722912073, -0.09713146090507507, 0.13070526719093323, 0.03612416610121727, -0.010269527323544025, 0.0973726138472557, 0.0160426814109087, -0.015357916243374348, -0.04583928361535072, 0.013540844433009624, 0.02855619788169861, 0.22257955372333527, 0.010157236829400063, -0.019154531881213188, 0.047470610588788986, -0.0663219541311264, 0.012325904332101345, 0.04028000682592392, 0.00878076907247305, 0.021861443296074867, -0.006834378466010094, 0.08050832152366638, -0.013388912193477154, 0.010295504704117775, -0.12394414097070694, 0.038067642599344254, 0.04333467409014702, -0.016984349116683006, 0.054271385073661804, -0.07370338588953018, -0.06158242002129555, -0.23119346797466278, -0.06624805182218552, -0.059147316962480545, 0.07341159135103226, -0.08091764152050018, -0.03261746093630791, 0.007119800429791212, 0.023471413180232048, 0.08402540534734726, 0.06494870781898499, -0.02099478989839554, -0.0158502496778965, -0.00767956068739295, -0.0008462801924906671, -0.0017221843590959907, -0.01417530607432127, 0.2016846090555191, -0.13482172787189484, -0.003628405975177884, 0.12596160173416138, -0.10128939151763916, 0.4350678026676178, 0.03538717329502106, -0.01763313077390194, -0.030514540150761604, 0.10444339364767075, 0.026718495413661003, 0.04950328171253204, 0.04966531693935394, -0.027433866634964943, 0.016581125557422638, -0.033761169761419296, 0.010837229900062084, 0.07848581671714783, 0.04351836442947388, -0.017385849729180336, 0.0549485869705677, -0.01000202540308237, -0.04975428059697151, 0.03728384152054787, -0.044196706265211105, 0.09179370850324631, 0.025426922366023064, 0.0032931140158325434, 0.0025982034858316183, 0.07077185064554214, -0.021295711398124695, 0.05649000033736229, -0.06738262623548508, 0.011861146427690983, 0.025450697168707848, 0.03521687164902687, 0.02107560820877552, 0.032688286155462265, -0.08445662260055542, -0.007823922671377659, 0.06207305192947388, 0.086041659116745, -0.008761605247855186, 0.004408304579555988, 0.06249147653579712, 0.05460096895694733, -0.058991480618715286, 0.028301214799284935, -0.04357035830616951, -0.05174628272652626, -0.02800653874874115, -0.06624153256416321, 0.034980658441782, -0.05637567117810249, -0.23873411118984222, 0.01997804082930088, 0.1021585464477539, 0.025300070643424988, -0.046574026346206665, 0.021609395742416382, -0.08211826533079147, 0.029910659417510033, -0.016428686678409576, -0.010845858603715897, 0.052116796374320984,

In [0]:
%sql
SELECT * FROM mvp.bronze.classifications
LIMIT 10

_id,attributes,incidents,namespace,notes,publish,created_at
60dd466180935bc89e6f9b14,"List(Map(short_name -> Annotator, value_json -> ""2""), Map(short_name -> Annotation Status, value_json -> ""6. Complete and final""), Map(short_name -> Reviewer, value_json -> ""6""), Map(short_name -> Quality Control, value_json -> false), Map(short_name -> Full Description, value_json -> ""Google-owned directions app Waze sent drivers toward areas impacted by the Skirball wildfires in Los Angeles late December 2017. The app looks at current traffic patterns and suggests routes that avoid major congestion. In the case of mass evacuations, as were implemented for these fires, congestion was seen on the evacuation routes leading the app to direct drivers toward the empty roads. These roads were empty because the area ablaze and impassable. Waze/Google engineers typically work with departments of transportation on traffic pattern changes to augment its directions, however in the case of a quickly-developed emergency, the app did not provide safe driving directions.""), Map(short_name -> Short Description, value_json -> ""Waze, a Google-owned directions app, led California drivers into the 2017 Skirball wildfires as they tried to evacuate the area.""), Map(short_name -> Beginning Date, value_json -> ""12-2017""), Map(short_name -> Ending Date, value_json -> ""12-2017""), Map(short_name -> Location, value_json -> ""Los Angeles, CA""), Map(short_name -> Near Miss, value_json -> ""Near miss""), Map(short_name -> Named Entities, value_json -> [""Waze"",""Google"",""Los Angeles Department of Transportation""]), Map(short_name -> Technology Purveyor, value_json -> [""Waze"",""Google""]), Map(short_name -> Intent, value_json -> ""Accident""), Map(short_name -> Severity, value_json -> ""Negligible""), Map(short_name -> Harm Type, value_json -> [""Harm to physical health/safety""]), Map(short_name -> Lives Lost, value_json -> false), Map(short_name -> Harm Distribution Basis, value_json -> []), Map(short_name -> Infrastructure Sectors, value_json -> [""Transportation""]), Map(short_name -> Financial Cost, value_json -> """"), Map(short_name -> Laws Implicated, value_json -> [""""]), Map(short_name -> AI System Description, value_json -> ""Waze, a directions app meant to decide what route from point A to point B would allow the shortest travel time. Waze also shows potential car accidents, police activity, and other hazards to consider while driving.""), Map(short_name -> Data Inputs, value_json -> [""Traffic patterns"",""human operator input""]), Map(short_name -> System Developer, value_json -> [""Google""]), Map(short_name -> Sector of Deployment, value_json -> [""Transportation and storage""]), Map(short_name -> Public Sector Deployment, value_json -> false), Map(short_name -> Nature of End User, value_json -> ""Amateur""), Map(short_name -> Level of Autonomy, value_json -> ""Medium""), Map(short_name -> Relevant AI functions, value_json -> [""Perception"",""Cognition""]), Map(short_name -> AI Techniques, value_json -> [""Interpreting traffic patterns"",""travel time estimation""]), Map(short_name -> AI Applications, value_json -> [""Direction suggestions"",""recommendation engine"",""""]), Map(short_name -> Physical System, value_json -> [""Software only""]), Map(short_name -> Problem Nature, value_json -> [""Robustness""]))",List(22),CSETv0,,true,null
60dd466280935bc89e6f9b1b,"List(Map(short_name -> Annotator, value_json -> ""2""), Map(short_name -> Annotation Status, value_json -> ""6. Complete and final""), Map(short_name -> Reviewer, value_json -> ""2""), Map(short_name -> Quality Control, value_json -> false), Map(short_name -> Full Description, value_json -> ""On May 6, 2010, the New York Stock Exchange and US Down Jones were greatly impacted by highly volatile trading at high volumes. The blame fell on a single trader in the UK, Navinder Singh Saroa, who allegedly modified a trading algorithm to allow him to mislead the market. Saroa would pl

# Camada Prata

Na camada prata é feita a limpeza e tratamento de qualidade dos dados, com exclusão de atributos que não serão usados no modelo final.

Na sequência, serão realizadas as cópias das tabelas, trazendo apenas os atributos de interesse já na etapa de seleção, e as inspeções/tratamentos de qualidade.
Serão usados recursos do Spark dataframes.

O código acima fez a cópia das tabelas para a camada prata trazendo apenas os atributos de interesse.

Na sequência, será realizado o tratamento de qualidade usando Spark dataframes.

## Tabela *incidents*

In [0]:
# Carrega a tabela da camada bronze apenas com os atributos de interesse
df = spark.table("mvp.bronze.incidents").select(
    "_id",
    "incident_id",
    "date",
    "title",
    "description",
    "reports",
    "Alleged_deployer_of_AI_system",
    "Alleged_developer_of_AI_system",
    "Alleged_harmed_or_nearly_harmed_parties",
    "implicated_systems",
    "tsne",
)

# Informações básicas incluindo contagem de nulos e tipos de cada atributo
df.summary("count", "min", "max").show()
print("Contagem de nulos:")
df.select([F.count(F.when(F.col(c).isNull(), 1)).alias(c) for c in df.columns]).show()
print("Tipos das colunas:")
for k, v in df.dtypes:
    print(f'- "{k}":  {v}')

+-------+--------------------+-----------+----------+--------------------+--------------------+
|summary|                 _id|incident_id|      date|               title|         description|
+-------+--------------------+-----------+----------+--------------------+--------------------+
|  count|                1294|       1294|      1294|                1294|                1294|
|    min|625763db343edc875...|          1|1983-09-26|"Jewish Baby Stro...| A Starship food ...|
|    max|693ccb74e03943780...|       1305|2025-12-11|“Amazon’s Choice”...|xAI's model Grok,...|
+-------+--------------------+-----------+----------+--------------------+--------------------+

Contagem de nulos:
+---+-----------+----+-----+-----------+-------+-----------------------------+------------------------------+---------------------------------------+------------------+----+
|_id|incident_id|date|title|description|reports|Alleged_deployer_of_AI_system|Alleged_developer_of_AI_system|Alleged_harmed_or_nearly_

In [0]:
# Converte 'date' para tipo date
df = df.withColumn("date", F.to_date(F.col("date")))

# Salva na camada silver
df.write.format("delta").mode("overwrite").saveAsTable("mvp.silver.incidents")

A seleção dos atributos de interesse foi realizada durante o carregamento da tabela no formato dataframe.
Apenas uma melhoria de qualidade foi identificada: conversão do atributo *date* do tipo *string* para *date*.

Segue extrato da tabela *incidents* salva na camada prata:

In [0]:
%sql
SELECT * FROM mvp.silver.incidents
LIMIT 5

_id,incident_id,date,title,description,reports,Alleged_deployer_of_AI_system,Alleged_developer_of_AI_system,Alleged_harmed_or_nearly_harmed_parties,implicated_systems,tsne
625763de343edc875fe63a15,23,2017-11-08,Las Vegas Self-Driving Bus Involved in Accident,"A self-driving public shuttle by Keolis North America and Navya was involved in a collision with a human-driven delivery truck in Las Vegas, Nevada on its first day of service.","List(242, 243, 244, 245, 246, 247, 248, 249, 250, 253, 254, 257, 258, 259, 260, 261, 263, 264, 266, 267, 268, 269, 270, 2389)","List(navya, keolis-north-america)","List(navya, keolis-north-america)","List(navya, keolis-north-america, bus-passengers)",List(),"Map(x -> -0.4086583321674232, y -> -0.7338369502943811)"
625763dc343edc875fe63a02,4,2018-03-18,Uber AV Killed Pedestrian in Arizona,"An Uber autonomous vehicle (AV) in autonomous mode struck and killed a pedestrian in Tempe, Arizona.","List(629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 644, 645, 646, 647, 1375, 1376, 1377, 1378, 1542, 2147, 1257)",List(uber),List(uber),"List(elaine-herzberg, pedestrians)",List(),"Map(x -> -0.3819100177468858, y -> -0.7647276622104244)"
625763db343edc875fe639ff,1,2015-05-19,Google’s YouTube Kids App Presents Inappropriate Content,YouTube’s content filtering and recommendation algorithms exposed children to disturbing and inappropriate videos.,"List(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15)",List(youtube),List(youtube),List(children),List(),"Map(x -> 0.1440729050145679, y -> -0.3860277814216216)"
625763de343edc875fe63a10,18,2015-04-04,Gender Biases of Google Image Search,"Google Image returns results that under-represent women in leadership roles, notably with the first photo of a female ""CEO"" being a Barbie doll after 11 rows of male CEOs.","List(130, 131, 132, 133, 134, 135, 136, 137, 138, 1367, 1368)",List(google),List(google),List(women),List(),"Map(x -> 0.05616459256769205, y -> -0.5115127448515291)"
625763dd343edc875fe63a0a,12,2016-07-21,Common Biases of Vector Embeddings,"Researchers from Boston University and Microsoft Research, New England demonstrated gender bias in the most common techniques used to embed words for natural language processing (NLP).",List(42),"List(microsoft-research, boston-university)","List(microsoft-research, google, boston-university)","List(women, minority-groups)",List(),"Map(x -> 0.16537207868368234, y -> -0.8452564671999145)"


## Tabela *classifications*

In [0]:
# Carrega a tabela da camada bronze apenas com os atributos de interesse
df = spark.table("mvp.bronze.classifications").select(
    "_id", "incidents", "namespace", "attributes", "notes", "publish"
)

# Informações básicas incluindo contagem de nulos e tipos de cada atributo
df.summary("count", "min", "max").show()
print("Contagem de nulos:")
df.select([F.count(F.when(F.col(c).isNull(), 1)).alias(c) for c in df.columns]).show()
print("Tipos das colunas:")
for k, v in df.dtypes:
    print(f'- "{k}":  {v}')

+-------+--------------------+---------+-----+
|summary|                 _id|namespace|notes|
+-------+--------------------+---------+-----+
|  count|                2367|     2367| 2367|
|    min|60dd465f80935bc89...|   CSETv0|     |
|    max|69362527d99435ade...|      MIT|   {}|
+-------+--------------------+---------+-----+

Contagem de nulos:
+---+---------+---------+----------+-----+-------+
|_id|incidents|namespace|attributes|notes|publish|
+---+---------+---------+----------+-----+-------+
|  0|        0|        0|         0|    0|      0|
+---+---------+---------+----------+-----+-------+

Tipos das colunas:
- "_id":  string
- "incidents":  array<bigint>
- "namespace":  string
- "attributes":  array<map<string,string>>
- "notes":  string
- "publish":  boolean


In [0]:
# Salva na camada silver
df.write.format("delta").mode("overwrite").saveAsTable("mvp.silver.classifications")

A seleção dos atributos de interesse foi realizada durante o carregamento da tabela no formato dataframe.
Nenhuma melhoria de qualidade foi necessária nessa etapa.

Segue extrato da tabela *classifications* salva na camada prata:

In [0]:
%sql
SELECT * FROM mvp.silver.classifications
LIMIT 5

_id,incidents,namespace,attributes,notes,publish
60dd466180935bc89e6f9b14,List(22),CSETv0,"List(Map(short_name -> Annotator, value_json -> ""2""), Map(short_name -> Annotation Status, value_json -> ""6. Complete and final""), Map(short_name -> Reviewer, value_json -> ""6""), Map(short_name -> Quality Control, value_json -> false), Map(short_name -> Full Description, value_json -> ""Google-owned directions app Waze sent drivers toward areas impacted by the Skirball wildfires in Los Angeles late December 2017. The app looks at current traffic patterns and suggests routes that avoid major congestion. In the case of mass evacuations, as were implemented for these fires, congestion was seen on the evacuation routes leading the app to direct drivers toward the empty roads. These roads were empty because the area ablaze and impassable. Waze/Google engineers typically work with departments of transportation on traffic pattern changes to augment its directions, however in the case of a quickly-developed emergency, the app did not provide safe driving directions.""), Map(short_name -> Short Description, value_json -> ""Waze, a Google-owned directions app, led California drivers into the 2017 Skirball wildfires as they tried to evacuate the area.""), Map(short_name -> Beginning Date, value_json -> ""12-2017""), Map(short_name -> Ending Date, value_json -> ""12-2017""), Map(short_name -> Location, value_json -> ""Los Angeles, CA""), Map(short_name -> Near Miss, value_json -> ""Near miss""), Map(short_name -> Named Entities, value_json -> [""Waze"",""Google"",""Los Angeles Department of Transportation""]), Map(short_name -> Technology Purveyor, value_json -> [""Waze"",""Google""]), Map(short_name -> Intent, value_json -> ""Accident""), Map(short_name -> Severity, value_json -> ""Negligible""), Map(short_name -> Harm Type, value_json -> [""Harm to physical health/safety""]), Map(short_name -> Lives Lost, value_json -> false), Map(short_name -> Harm Distribution Basis, value_json -> []), Map(short_name -> Infrastructure Sectors, value_json -> [""Transportation""]), Map(short_name -> Financial Cost, value_json -> """"), Map(short_name -> Laws Implicated, value_json -> [""""]), Map(short_name -> AI System Description, value_json -> ""Waze, a directions app meant to decide what route from point A to point B would allow the shortest travel time. Waze also shows potential car accidents, police activity, and other hazards to consider while driving.""), Map(short_name -> Data Inputs, value_json -> [""Traffic patterns"",""human operator input""]), Map(short_name -> System Developer, value_json -> [""Google""]), Map(short_name -> Sector of Deployment, value_json -> [""Transportation and storage""]), Map(short_name -> Public Sector Deployment, value_json -> false), Map(short_name -> Nature of End User, value_json -> ""Amateur""), Map(short_name -> Level of Autonomy, value_json -> ""Medium""), Map(short_name -> Relevant AI functions, value_json -> [""Perception"",""Cognition""]), Map(short_name -> AI Techniques, value_json -> [""Interpreting traffic patterns"",""travel time estimation""]), Map(short_name -> AI Applications, value_json -> [""Direction suggestions"",""recommendation engine"",""""]), Map(short_name -> Physical System, value_json -> [""Software only""]), Map(short_name -> Problem Nature, value_json -> [""Robustness""]))",,true
60dd466280935bc89e6f9b1b,List(28),CSETv0,"List(Map(short_name -> Annotator, value_json -> ""2""), Map(short_name -> Annotation Status, value_json -> ""6. Complete and final""), Map(short_name -> Reviewer, value_json -> ""2""), Map(short_name -> Quality Control, value_json -> false), Map(short_name -> Full Description, value_json -> ""On May 6, 2010, the New York Stock Exchange and US Down Jones were greatly impacted by highly volatile trading at high volumes. The blame fell on a single trader in the UK, Navinder Singh Saroa, who allegedly modified a trading algorithm to allow him to mislead the market. Saroa would pl

## Tabela *reports*

In [0]:
# Carrega a tabela da camada bronze apenas com os atributos de interesse
df = spark.table("mvp.bronze.reports").select(
    "_id", "report_number", "date_published", "title", "description", "source_domain"
)

# Informações básicas incluindo contagem de nulos e tipos de cada atributo
df.summary("count", "min", "max").show()
print("Contagem de nulos:")
df.select([F.count(F.when(F.col(c).isNull(), 1)).alias(c) for c in df.columns]).show()
print("Tipos das colunas:")
for k, v in df.dtypes:
    print(f'- "{k}":  {v}')

+-------+--------------------+-------------+-------------------------------------+--------------------+-----------------+
|summary|                 _id|report_number|                                title|         description|    source_domain|
+-------+--------------------+-------------+-------------------------------------+--------------------+-----------------+
|  count|                6554|         6554|                                 6554|                2020|             6554|
|    min|5d34b8c29ced494f0...|            1|                                     |                    |                 |
|    max|693da351678c7ec53...|         6745|ＮＨＫ、ＡＩの多言語字幕サービス終...|😱 Wait! What? Ju...|zive.aktuality.sk|
+-------+--------------------+-------------+-------------------------------------+--------------------+-----------------+

Contagem de nulos:
+---+-------------+--------------+-----+-----------+-------------+
|_id|report_number|date_published|title|description|source_domain|
+---+-----

In [0]:
# Salva na camada silver
df.write.format("delta").mode("overwrite").saveAsTable("mvp.silver.reports")

A seleção dos atributos de interesse foi realizada durante o carregamento da tabela no formato dataframe.
Nenhuma melhoria de qualidade foi necessária nessa etapa.

Segue extrato da tabela *reports* salva na camada prata:

In [0]:
%sql
SELECT * FROM mvp.silver.reports
LIMIT 5

_id,report_number,date_published,title,description,source_domain
5d34b8c29ced494f010ed45a,1,2015-05-19T00:00:00.000Z,Google’s YouTube Kids App Criticized for ‘Inappropriate Content’,"Child and consumer advocacy groups complained to the FTC that Google’s new YouTube Kids app contains “inappropriate content,” including explicit sexual language and jokes about pedophilia.",blogs.wsj.com
5d34b8c29ced494f010ed461,8,2018-04-26T00:00:00.000Z,"YouTube Kids, Criticized for Content, Introduces New Parental Controls","Parents will be able to handpick the channels and topics their children can view on the app, which has been criticized for allowing disturbing content to slip through.",nytimes.com
5d34b8c29ced494f010ed464,11,2018-03-23T00:00:00.000Z,"Children's YouTube is still churning out blood, suicide and cannibalism","Children's search terms on YouTube are still awash with bizarre and sometimes disturbing content. Why, and can anything be done to stem the tide?",wired.co.uk
5d34b8c29ced494f010ed45b,2,2018-02-07T00:00:00.000Z,YouTube Kids app is STILL showing disturbing videos,"Investigators found several unsuitable videos on the San Bruno-based site, including one of a burning aeroplane from the cartoon Paw Patrol (pictured).",dailymail.co.uk
5d34b8c29ced494f010ed462,9,2018-03-17T00:00:00.000Z,YouTube suggested conspiracy videos to children using its Kids app,YouTube removed videos from conspiracy theorist David Icke after we contacted the company.,businessinsider.com


## Tabela *entities*

In [0]:
# Carrega a tabela da camada bronze apenas com os atributos de interesse
df = spark.table("mvp.bronze.entities").select("_id", "entity_id", "name")

# Informações básicas incluindo contagem de nulos e tipos de cada atributo
df.summary("count", "min", "max").show()
print("Contagem de nulos:")
df.select([F.count(F.when(F.col(c).isNull(), 1)).alias(c) for c in df.columns]).show()
print("Tipos das colunas:")
for k, v in df.dtypes:
    print(f'- "{k}":  {v}')

+-------+--------------------+---------------+----+
|summary|                 _id|      entity_id|name|
+-------+--------------------+---------------+----+
|  count|                5277|           5277|5277|
|    min|636f26b6343758cf8...|               |    |
|    max|693ccb5ed99435ade...|zuzana-caputova|zlib|
+-------+--------------------+---------------+----+

Contagem de nulos:
+---+---------+----+
|_id|entity_id|name|
+---+---------+----+
|  0|        0|   0|
+---+---------+----+

Tipos das colunas:
- "_id":  string
- "entity_id":  string
- "name":  string


In [0]:
# Salva na camada silver
df.write.format("delta").mode("overwrite").saveAsTable("mvp.silver.entities")

A seleção dos atributos de interesse foi realizada durante o carregamento da tabela no formato dataframe.
Nenhuma melhoria de qualidade foi necessária nessa etapa.

Segue extrato da tabela *entities* salva na camada prata:

In [0]:
%sql
SELECT * FROM mvp.silver.entities
LIMIT 5

_id,entity_id,name
636f26b7343758cf81522afd,amazon,Amazon
636f26b7343758cf81522b03,elaine-herzberg,Elaine Herzberg
636f26b7343758cf81522b06,doctors,Doctors
636f26b8343758cf81522b2a,navinder-sarao,Navinder Sarao
636f26b7343758cf81522b01,airplane-crew,Airplane Crew


# Camada Ouro

Nesta camada geramos as tabelas finais do modelo, que serão utilizadas para fazer a análise do problema.

In [0]:
%sql
CREATE OR REPLACE TABLE mvp.gold.incidents_classified AS
WITH filtered_classifications AS (
  SELECT
    c.incidents,
    from_json(
      lower(get(filter(c.attributes, x ->
        lower(x['short_name']) = 'harm distribution basis' AND trim(x['value_json']) <> ''
      ), 0)['value_json']),
      'array<string>'
    ) AS attr_harm_dist_basis,
    get(filter(c.attributes, x ->
      lower(x['short_name']) = 'lives lost' AND trim(x['value_json']) <> ''
    ), 0)['value_json'] AS attr_lives_lost
  FROM mvp.silver.classifications c
)
SELECT
  i.incident_id,
  i.date,
  i.title,
  i.Alleged_developer_of_AI_system,
  i.Alleged_harmed_or_nearly_harmed_parties,
  array_distinct(flatten(collect_list(f.attr_harm_dist_basis))) AS attr_harm_dist_basis,
  collect_set(f.attr_lives_lost) AS attr_lives_lost,
  max(CASE
    WHEN f.attr_harm_dist_basis IS NULL THEN NULL
    WHEN exists(attr_harm_dist_basis, x -> x LIKE '%race%' OR x LIKE '%sex%' OR x LIKE '%religion%') THEN TRUE
    ELSE FALSE
  END) AS social_bias,
  max(CASE
    WHEN f.attr_lives_lost IS NULL THEN NULL
    WHEN lower(f.attr_lives_lost) LIKE 'true' OR try_cast(f.attr_lives_lost AS INT) > 0 THEN TRUE
    ELSE FALSE
  END) AS lives_lost
FROM mvp.silver.incidents i
JOIN filtered_classifications f
  ON array_contains(f.incidents, i.incident_id)
GROUP BY
  i.incident_id,
  i.date,
  i.title,
  i.Alleged_developer_of_AI_system,
  i.Alleged_harmed_or_nearly_harmed_parties

num_affected_rows,num_inserted_rows


A nova tabela gerada acima consiste na tabela *incidents* enriquecida com algumas classificações chaves para as questões lançadas no objetivo.

Primeiramente filtramos a tabela *classifications* para retornar apenas a coluna *incidents*, que correlaciona as classificações com os incidentes, e duas variáveis extraídas da coluna original *attributes*, correspondentes às classificações "Harm Distribution Basis" e "Lives Lost" presentes originalmente em arrays de structs dentro da coluna *attributes*.

A variável *attr_harm_dist_basis*, correspondente à classificação "Harm Distribution Basis", indica como os danos estão potencialmente distribuídos desigualmente na sociedade, podendo conter valores como "geography", "ideology", "race", "religion", etc.

A variável *attr_lives_lost*, correspondente à classificação "Lives Lost", indica o número de mortes ligadas ao incidente ou, em alguns registros, apenas se houve ou não mortes.

Após esse filtro da tabela *classifications*, fazemos o join do resultado com a tabela *incidents*, selecionando desta apenas variáveis de interesse, como data, título e partes envolvidas.

No join também agregamos em arrays os atributos distintos relacionados a um mesmo incidente e criamos as novas colunas booleanas *social_bias* e *lives_lost*, que representam, respectivamente, se o incidente teve alguma classificação relacionada a racismo, sexismo, orientação sexual ou intolerância religiosa, e se houve morte de pessoas.

Com a tabela resultante, chamada *incidents_classified* e salva no esquema *gold*, será possível realizar as análises que objetivamos.

In [0]:
%sql
-- DICIONÁRIO DE DADOS
--
-- TABELA:
--
-- incidents_classified
COMMENT ON TABLE mvp.gold.incidents_classified IS
"This table contains classified incidents with social bias and lives lost indicators.";

-- COLUNAS:
--
-- incident_id
COMMENT ON COLUMN mvp.gold.incidents_classified.incident_id IS
"Unique identifier for each incident. Type: bigint";

-- date
COMMENT ON COLUMN mvp.gold.incidents_classified.date IS
"Date when the incident occurred or was reported. Type: date";

-- title
COMMENT ON COLUMN mvp.gold.incidents_classified.title IS
"Title or brief description of the incident. Type: string";

-- Alleged_developer_of_AI_system
COMMENT ON COLUMN mvp.gold.incidents_classified.Alleged_developer_of_AI_system IS
"List of alleged developers or organizations responsible for the AI system involved in the incident. Type: array<string>";

-- Alleged_harmed_or_nearly_harmed_parties
COMMENT ON COLUMN mvp.gold.incidents_classified.Alleged_harmed_or_nearly_harmed_parties IS
"List of parties allegedly harmed or nearly harmed in the incident. Type: array<string>";

-- attr_harm_dist_basis
COMMENT ON COLUMN mvp.gold.incidents_classified.attr_harm_dist_basis IS
"List of harm distribution basis values extracted from classification attributes. Type: array<string>";

-- attr_lives_lost
COMMENT ON COLUMN mvp.gold.incidents_classified.attr_lives_lost IS
"List of lives lost values extracted from classification attributes. Type: array<string>";

-- social_bias
COMMENT ON COLUMN mvp.gold.incidents_classified.social_bias IS
"Indicates if the incident has social bias based on harm distribution basis. Type: boolean";

-- lives_lost
COMMENT ON COLUMN mvp.gold.incidents_classified.lives_lost IS
"Indicates if the incident resulted in lives lost. Type: boolean";

A query acima consolida o dicionário de dados para a tabela recém criada na camada ouro.

Segue abaixo um extrato da tabela *incidents_classified*:

In [0]:
%sql
SELECT * FROM mvp.gold.incidents_classified
LIMIT 10

incident_id,date,title,Alleged_developer_of_AI_system,Alleged_harmed_or_nearly_harmed_parties,attr_harm_dist_basis,attr_lives_lost,social_bias,lives_lost
1,2015-05-19,Google’s YouTube Kids App Presents Inappropriate Content,List(youtube),List(children),"List(none, age)","List(0, false)",false,false
2,2018-12-05,Warehouse robot ruptures can of bear spray and injures workers,List(amazon),List(warehouse-workers),List(none),"List(0, false)",false,false
3,2018-10-27,Crashes with Maneuvering Characteristics Augmentation System (MCAS),List(boeing),"List(airplane-passengers, airplane-crew)",List(none),"List(189, true)",false,true
4,2018-03-18,Uber AV Killed Pedestrian in Arizona,List(uber),"List(elaine-herzberg, pedestrians)",List(none),"List(1, true)",false,true
5,2015-07-13,Collection of Robotic Surgery Malfunctions,List(intuitive-surgical),List(patients),List(none),"List(144, true)",false,true
6,2016-03-24,"Microsoft's TayBot Allegedly Posts Racist, Sexist, and Anti-Semitic Content to Twitter",List(microsoft),List(twitter-users),"List(race, religion, sex, ideology, nation of origin, citizenship, immigrant status, national origin or immigrant status)","List(0, false)",true,false
7,2017-02-24,Wikipedia Vandalism Prevention Bot Loop,List(wikipedia),"List(wikimedia-foundation, wikipedia-editors, wikipedia-users)",List(none),"List(0, false)",false,false
8,2014-08-15,Uber Autonomous Cars Running Red Lights,List(uber),"List(pedestrians, motorists)",List(none),"List(0, false)",false,false
9,2012-02-25,NY City School Teacher Evaluation Algorithm Contested,List(new-york-city-dept.-of-education),List(teachers),"List(none, other:school teachers)","List(0, false)",false,false
10,2014-08-14,Kronos Scheduling Algorithm Allegedly Caused Financial Issues for Starbucks Employees,List(kronos),List(starbucks-employees),List(none),"List(0, false)",false,false


# Análise

Nesta etapa responderemos às perguntas traçadas inicialmente no objetivo.
Para tal, faremos operações em cima da tabela *incidents_classified* criada na camada ouro.

In [0]:
%sql
SELECT COUNT(*) AS num_incidents_with_lives_lost
FROM mvp.gold.incidents_classified
WHERE lives_lost = TRUE;

num_incidents_with_lives_lost
22


**1. Existem incidentes de IA registrados que envolvem morte de pessoas?**

Sim. Com a query acima identificamos, dentre os registros analisados, **22 incidentes com morte** em que a IA teve envolvimento na causa.

In [0]:
%sql
SELECT COUNT(*) AS num_incidents_with_social_bias
FROM mvp.gold.incidents_classified
WHERE social_bias = TRUE;

num_incidents_with_social_bias
91


**2. Existem incidentes registrados que evidenciam que os modelos de IA absorvem problemas de viés social, como racismo, homofobia, intolerância religiosa e preconceitos em geral?**

Sim. Podemos ver que **91 incidentes** dentre os registros tiveram algum viés social envolvendo **racismo, sexismo, homofobia ou intolerância religiosa**.

In [0]:
%sql
SELECT
  COUNT(*) AS total_incidents,
  SUM(CASE WHEN lives_lost = TRUE THEN 1 ELSE 0 END) AS incidents_with_lives_lost,
  SUM(CASE WHEN social_bias = TRUE THEN 1 ELSE 0 END) AS incidents_with_social_bias,
  ROUND(100.0 * SUM(CASE WHEN lives_lost = TRUE THEN 1 ELSE 0 END) / COUNT(*), 2) AS pct_lives_lost_total,
  ROUND(100.0 * SUM(CASE WHEN social_bias = TRUE THEN 1 ELSE 0 END) / COUNT(*), 2) AS pct_social_bias_total,
  ROUND(100.0 * SUM(CASE WHEN lives_lost = TRUE THEN 1 ELSE 0 END) / COUNT(lives_lost), 2) AS pct_lives_lost_classified,
  ROUND(100.0 * SUM(CASE WHEN social_bias = TRUE THEN 1 ELSE 0 END) / COUNT(social_bias), 2) AS pct_social_bias_classified
FROM mvp.gold.incidents_classified

total_incidents,incidents_with_lives_lost,incidents_with_social_bias,pct_lives_lost_total,pct_social_bias_total,pct_lives_lost_classified,pct_social_bias_classified
1242,22,91,1.77,7.33,6.88,28.44


**3. Qual é a representatividade desses tipos de incidente dentre os registros encontrados?**

Com base na query realizada acima temos que:
- 1,8% do total de incidentes envolvem morte de pessoas
- 6,9% do total de incidentes classificados envolvem morte de pessoas
- 7,3% do total de incidentes envolvem problemas de viés social
- 28,4% do total de incidentes classificados envolvem problemas de viés social

São números expressivos, considerando a base que estamos analisando.
O percentual aumenta quando colocamos no denominador apenas o total de registros devidamente classificados.
A representatividade real para a amostra, portanto, deve ficar **entre 1,8% e 6,9% para incidentes com mortes** e **entre 7,3% e 28,4% para incidentes com problema de viés social**.

Logicamente a amostra não é grande e existe um erro a ser considerado devido a isso.
Outros fatores como vieses nas publicações/notificações, erros de classificação e imprecisões na metodologia da fonte original dos dados em termos de coleta e tratamento também devem ser levados em conta.

In [0]:
# Carrega a tabela de incidentes da camada gold
df = spark.table("mvp.gold.incidents_classified").select("incident_id", "date")

# Agrupa por ano e conta incidentes
df_year = (
    df.withColumn("year", F.year(F.col("date")))
    .groupBy("year")
    .count()
    .orderBy("year")
)

# Exibe gráfico de linha
display(df_year)

year,count
1983,1
1992,1
1996,1
1998,1
1999,1
2003,4
2006,1
2007,1
2008,3
2009,2


Databricks visualization. Run in Databricks to view.

**4. A taxa de ocorrência de incidentes de IA vem aumentando nos últimos anos?**

Através do gráfico acima notamos que a quantidade de incidentes de IA reportados permaneceu baixa e estável até 2013.
Após esse ano, teve um aumento aproximadamente linear até 2021.
De 2021 em diante a taxa de incidentes aumentou substancialmente.

Esse comportamento está em linha com o surgimento e avanços das tecnologias e algoritmos nos anos recentes e mostra a importância de se ter uma regulação sobre certos aspectos da IA, dado que ela vem sendo cada vez mais usada, influenciando cada vez mais em nossas vidas e, portanto, com aumento gradual da nossa exposição à possíveis falhas.

# Conclusão

A análise realizada permitiu identificar padrões relevantes sobre incidentes envolvendo inteligência artificial, revelando tanto riscos diretos à vida humana quanto problemas estruturais ligados a vieses sociais presentes nos modelos. Entre os registros analisados, foram encontrados 22 incidentes com morte ligada ao uso de IA e 91 incidentes relacionados a manifestações de racismo, sexismo, homofobia ou intolerância religiosa. A representatividade desses casos, quando considerada dentro do total geral e dentro apenas dos incidentes classificados, variou entre 1,8% e 6,9% para mortes e entre 7,3% e 28,4% para incidentes com viés social, números expressivos diante do tamanho da amostra e das limitações metodológicas. Além disso, observou-se um crescimento significativo na taxa de incidentes ao longo do tempo, especialmente a partir de 2021, alinhado com a expansão do uso da IA e a crescente integração desses sistemas em atividades sensíveis e de alto impacto. Esses achados reforçam a necessidade de políticas de governança, mecanismos de mitigação de riscos e regulações adequadas para acompanhar a evolução acelerada dessa tecnologia.

# Autoavaliação

O trabalho desenvolvido alcançou resultados relevantes dentro das condições e limitações da análise, oferecendo um panorama inicial consistente sobre os incidentes envolvendo IA. A investigação conseguiu identificar casos significativos, incluindo incidentes com perda de vidas humanas e exemplos de vieses sociais reproduzidos por modelos algorítmicos, fornecendo uma visão concreta dos riscos já observados. A quantificação da representatividade desses eventos, embora sujeita a incertezas e à limitação da amostra disponível, contribui para uma compreensão mais estruturada do problema. Da mesma forma, o padrão de crescimento dos incidentes ao longo dos anos reforça a pertinência do tema e a utilidade do estudo para orientar discussões futuras. Assim, mesmo como um esforço inicial, o trabalho oferece uma base válida sobre a qual análises mais completas e aprofundadas poderão ser desenvolvidas.

Do ponto de vista de autoavaliação do trabalho, há diversas oportunidades de aprimoramento metodológico e analítico. Em primeiro lugar, o processo de classificação dos incidentes poderia ser significativamente enriquecido ao empregar modelos de linguagem (LLMs) para leitura e interpretação direta das descrições e dos reports vinculados a cada caso. A abordagem atual, baseada sobretudo nas taxonomias pré-existentes — como a categoria “harm distribution basis” — depende da precisão, consistência e completude das classificações fornecidas pela fonte original dos dados. Embora essa taxonomia ofereça uma padronização mínima, ela pode deixar lacunas, seja por rotulagem incompleta, seja por classificações ambíguas ou inadequadas. O uso de LLMs permitiria conferir maior profundidade, sensibilidade e granularidade à análise, identificando nuances que os rótulos estruturados podem não capturar, como contexto social, causas indiretas, fatores sistêmicos e indícios de viés não explicitamente rotulados. Além disso, permitiria uma auditoria cruzada: comparar a classificação automática baseada em narrativa com a classificação declarada, detectando inconsistências e reforçando a confiabilidade dos resultados.

Outro aspecto importante seria a ampliação das fontes de dados utilizadas. O repositório analisado fornece uma visão útil, porém limitada, e potencialmente enviesada pelo método de coleta ou pela natureza das publicações de incidentes. A incorporação de bases adicionais — como o banco de incidentes mantido pela OECD (https://oecd.ai/incidents) e outras iniciativas internacionais de monitoramento de falhas de IA — traria maior diversidade de contextos, setores e geografias, reduzindo vieses de amostragem e aumentando a representatividade estatística. Além disso, permitiria avaliar se os padrões observados são consistentes entre diferentes bases, contribuindo para conclusões mais robustas e generalizáveis. Uma estratégia complementar seria padronizar procedimentos de limpeza, deduplicação e harmonização entre diferentes repositórios, garantindo maior consistência no tratamento dos dados.

Por fim, também seria possível aprimorar a análise temporal com métodos estatísticos mais sofisticados, permitindo avaliar tendências com maior precisão e distinguir entre aumentos reais na frequência de incidentes e aumentos decorrentes de fatores como maior transparência, intensificação da atenção pública ou o simples crescimento no número de sistemas de IA em operação. Além disso, a realização de levantamentos estratificados — segmentando os incidentes por região geográfica, tipo de dano, área de aplicação (como financeiro, segurança, indústria, transporte, tecnologia, entre outras), ou outras classificações relevantes — ampliaria significativamente a capacidade de identificar padrões específicos, concentrações de risco e vulnerabilidades sistêmicas que podem estar ocultas nas estatísticas agregadas. Técnicas de modelagem de séries temporais, intervalos de confiança robustos, análises de sensibilidade e métodos de correção de vieses de amostragem complementariam esse aprofundamento, oferecendo uma visão mais acurada e confiável do comportamento dos incidentes ao longo do tempo.

A adoção dessas melhorias tornaria o estudo mais completo, rigoroso e útil para embasar discussões sobre governança, regulação e mitigação de riscos associados à inteligência artificial.